# Protocol

This protocol creates a 2d protein & salt gradient. In particular, it manages the following things:
- it distributes two buffers in a specified linear gradient across columns of a 384-well plate
- it pipettes protein into well A1 of the plate
- it serially dilutes the protein across specified rows
- it distributes the different dilutions into the different salt concentrations 
- slow mixing in the end.


First we need to import some packages.

In [1]:
from opentrons import simulate, execute
from opentrons.types import Location, Point

import numpy as np
import pandas as pd
import time

Load the protocol and home the robot:

In [2]:
protocol = execute.get_protocol_api('2.11')

protocol.home()

/data/robot_settings.json not found. Loading defaults
Failed to initialize character device, will not be able to control gpios (lights, button, smoothiekill, smoothie reset). Only one connection can be made to the gpios at a time. If you need to control gpios, first stop the robot server with systemctl stop opentrons-robot-server. Until you restart the server with systemctl start opentrons-robot-server, you will be unable to control the robot using the Opentrons app.
/data/deck_calibration.json not found. Loading defaults


If we want to simulate:

In [45]:
protocol = simulate.get_protocol_api('2.11')

protocol.home()

C:\Users\raras\.opentrons\robot_settings.json not found. Loading defaults
C:\Users\raras\.opentrons\deck_calibration.json not found. Loading defaults


### Load the labware with the following setup:

<table style="width:300px;">
    <tr><td>10</td><td>11</td><td>trash</td></tr>
    <tr><td>7</td><td>8</td><td>9</td></tr>
    <tr><td>4 <br> tips 20</td><td>5 <br> plate </td><td>6 <br> tube rack</td></tr>
    <tr><td>1 <br> tips 300</td><td>2 <br> reservoir</td><td>3</td></tr>
    
</table>

In [46]:
tips_300 = protocol.load_labware("opentrons_96_tiprack_300ul", 1)

reservoir = protocol.load_labware("nest_12_reservoir_15ml", 2)

tips_20 = protocol.load_labware("opentrons_96_filtertiprack_20ul", 4)

plate = protocol.load_labware("corning_384_wellplate_112ul_flat", 5)

tube_rack = protocol.load_labware("opentrons_10_tuberack_falcon_4x50ml_6x15ml_conical", 6)

### Load pipettes 

Specify the correct tip racks. Uncomment the pipette you attempt to use and comment the pipette that is not used.

If using the multi pipette set it to "True"

In [47]:
# pip_20 = protocol.load_instrument("p300_single_gen2", "left", tip_racks=[tips_300])
pip_20 = protocol.load_instrument("p20_single_gen2", "right", tip_racks=[tips_20])
pip_300  = protocol.load_instrument("p300_single_gen2", "left", tip_racks=[tips_300])


If necessary, specify starting tip:

In [48]:
pip_20.starting_tip = tips_20.well("A1")

### Load buffers:

#### Buffer A:

In [10]:
buffer_A = {"pos":reservoir["A1"]}
buffer_A["c"] = float(input("Starting concentration of buffer A in mM"))

Starting concentration of buffer A in mM 0


#### Buffer B:

In [11]:
buffer_B = {"pos":reservoir["A2"]}
buffer_B["c"] = float(input("Starting concentration of buffer B in mM"))

Starting concentration of buffer B in mM 500


Optional: Protein Buffer:

In [ ]:
# protein_buffer = {"pos":reservoir["A2"]}
# protein_buffer["description"] = "KCl"
# protein_buffer["conc"] = float(input(f'Starting concentration of protein buffer ({protein_buffer["description"]}) in mM'))
# protein_buffer["volume"] = float(input("Volume of protein buffer in ml")

In [12]:
max_volume_buffers = float(input("What is the maximum volume of buffer per well?"))
step_size_buffers = float(input("What decrease of buffer volume per well?"))
v_protein_to_wells = float(input("What amount of protein solution will be added in the end per well?"))

What is the maximum volume of buffer per well? 20
What decrease of buffer volume per well? 5
What amount of protein solution will be added in the end per well? 5


#### Protein:

In [13]:
protein = {"pos":tube_rack["A1"]}
protein["c0"] = float(input("Starting concentration of protein in µM"))
protein["c1"] = float(input("Desired end concentration of protein in µM"))
protein["n dilutions"] = int(input("Number of dilutions"))
protein["v1"] = float(input("End volume in each well"))

Starting concentration of protein in µM 35
Desired end concentration of protein in µM 0.035
Number of dilutions 10
End volume in each well 50


##### Serial Dilution of the Protein


$ c_{final} = \frac{v_{transfer}}{v_{transfer} + v_{buffer} }^{n} \cdot c_{0} $

=> 

$ v_{transfer} = \sqrt[n]{ \frac{c_{final}} {c_{0}} } \cdot v_{final} \cdot \frac{1} {\sqrt[n]{ \frac{c_{final}} {c_{0}} }-1} $

In [14]:
c_start = protein["c0"]
c_final = protein["c1"]
n = protein["n dilutions"]
v_final = protein["v1"]

In [15]:
protein["v_t"] = (c_final/c_start)**(1/n) * v_final / (1-(c_final/c_start)**(1/n) )
print(f'The transfer volume will be appr. {protein["v_t"]:.2f} µl')

The transfer volume will be appr. 50.24 µl


In [16]:
print("Concentrations are appr.:")
for i in range(0, n+1):
    c= (protein["v_t"]/(protein["v_t"]+v_final))**i * c_start   
    print(f'{c:.4f} µM')

Concentrations are appr.:
35.0000 µM
17.5416 µM
8.7916 µM
4.4062 µM
2.2084 µM
1.1068 µM
0.5547 µM
0.2780 µM
0.1393 µM
0.0698 µM
0.0350 µM


### Calculation of buffer volumes and concentrations per well

In [17]:
vols_A = np.arange(0, max_volume_buffers + step_size_buffers, step_size_buffers)
vols_A = np.sort(vols_A)[::-1]

vols_B = [20 - vol for vol in vols_A]

num_cols = len(vols_A)
print("Volumes will be (in descending order): ", vols_A)
if num_cols < len(plate.columns()):
    print(f'Okay, {num_cols} columns will be needed.')
else:
    raise ValueError('The plate does not have enough columns.')

Volumes will be (in descending order):  [20. 15. 10.  5.  0.]
Okay, 5 columns will be needed.


In [34]:
if pip_20.hw_pipette["name"].find("multi") > 0:
    n = 2
    print("hey")
else: 
    n = len(plate.rows())
vols_A_384 = list(np.repeat(vols_A, n))
vols_B_384 = [max_volume_buffers - vol_A for vol_A in vols_A_384]


# get first two rows. When using a multipipette we have to give the top well to the respective function. Because 
# we use a 384 well plate, we have to pass the top well to fill all oddly numbered rows and the second well from
# the top to fill all evenly numbered rows
wells_to_pipette_to = np.array([wells[0:n] for wells in plate.columns()[1:num_cols+1]]).flatten()

# convert back to list:
wells_to_pipette_to = list(wells_to_pipette_to)

print("The following wells will be filled:")
[print(well) for well in wells_to_pipette_to];

The following wells will be filled:
A2 of Corning 384 Well Plate 112 µL Flat on 5
B2 of Corning 384 Well Plate 112 µL Flat on 5
C2 of Corning 384 Well Plate 112 µL Flat on 5
D2 of Corning 384 Well Plate 112 µL Flat on 5
E2 of Corning 384 Well Plate 112 µL Flat on 5
F2 of Corning 384 Well Plate 112 µL Flat on 5
G2 of Corning 384 Well Plate 112 µL Flat on 5
H2 of Corning 384 Well Plate 112 µL Flat on 5
I2 of Corning 384 Well Plate 112 µL Flat on 5
J2 of Corning 384 Well Plate 112 µL Flat on 5
K2 of Corning 384 Well Plate 112 µL Flat on 5
L2 of Corning 384 Well Plate 112 µL Flat on 5
M2 of Corning 384 Well Plate 112 µL Flat on 5
N2 of Corning 384 Well Plate 112 µL Flat on 5
O2 of Corning 384 Well Plate 112 µL Flat on 5
P2 of Corning 384 Well Plate 112 µL Flat on 5
A3 of Corning 384 Well Plate 112 µL Flat on 5
B3 of Corning 384 Well Plate 112 µL Flat on 5
C3 of Corning 384 Well Plate 112 µL Flat on 5
D3 of Corning 384 Well Plate 112 µL Flat on 5
E3 of Corning 384 Well Plate 112 µL Flat on 

In [35]:
c_list = [buffer_B["c"] * vol_B / max_volume_buffers for vol_B in vols_B ]

print('Buffer concentrations will be')
[print(f'{c:.2f} mM in column {i}') for c, i in zip(c_list, range(2,num_cols+2, 1))];

Buffer concentrations will be
0.00 mM in column 2
125.00 mM in column 3
250.00 mM in column 4
375.00 mM in column 5
500.00 mM in column 6


That information tells us how much protein/buffer volume we need **at least** per well in the first column:

In [36]:
if (num_cols * v_protein_to_wells > protein["v1"]):
    print("Not enough protein available. Increase the final volume of protein in the first column or decrease the amount of protein added to the buffer at the very end")
else:
    print(f'With {protein["v1"]} µl of protein solution per source well (first column) we have enough to add {v_protein_to_wells} µl to {num_cols} columns')

With 50.0 µl of protein solution per source well (first column) we have enough to add 5.0 µl to 5 columns


# Actual Pipetting

### Buffer A:

Complex command:

In [49]:
if not pip_20.hw_pipette['has_tip']:
    pip_20.pick_up_tip()

t_start = time.time()    
    
pip_20.distribute(vols_A_384,
              buffer_A["pos"],
              wells_to_pipette_to,
              new_tip='never',
              touch_tip=True, 
              disposal_volume = 1,
             )
t_end = time.time()
print(f'Needed {t_end-t_start}')

# pip_20.drop()

Needed 1.9960556030273438


In [50]:
for line in protocol.commands(): 
    print(line)

Picking up tip from A1 of Opentrons 96 Filter Tip Rack 20 µL on 4
Distributing [20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] from A1 of NEST 12 Well Reservoir 15 mL on 2 to A2 of Corning 384 Well Plate 112 µL Flat on 5
Transferring [20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5

Simple command:

In [51]:
if not pip_20.hw_pipette['has_tip']:
    pip_20.pick_up_tip()

t_start = time.time() 

for i, vol  in enumerate(vols_A_384):
    print(wells_to_pipette_to[i], vol)
    
    pip_20.aspirate(vol, buffer_A["pos"])
    
    pip_20.dispense(vol, wells_to_pipette_to[i])
    
t_end = time.time()
print(f'Needed {t_end-t_start}')

A2 of Corning 384 Well Plate 112 µL Flat on 5 20.0
B2 of Corning 384 Well Plate 112 µL Flat on 5 20.0
C2 of Corning 384 Well Plate 112 µL Flat on 5 20.0
D2 of Corning 384 Well Plate 112 µL Flat on 5 20.0
E2 of Corning 384 Well Plate 112 µL Flat on 5 20.0
F2 of Corning 384 Well Plate 112 µL Flat on 5 20.0
G2 of Corning 384 Well Plate 112 µL Flat on 5 20.0
H2 of Corning 384 Well Plate 112 µL Flat on 5 20.0
I2 of Corning 384 Well Plate 112 µL Flat on 5 20.0
J2 of Corning 384 Well Plate 112 µL Flat on 5 20.0
K2 of Corning 384 Well Plate 112 µL Flat on 5 20.0
L2 of Corning 384 Well Plate 112 µL Flat on 5 20.0
M2 of Corning 384 Well Plate 112 µL Flat on 5 20.0
N2 of Corning 384 Well Plate 112 µL Flat on 5 20.0
O2 of Corning 384 Well Plate 112 µL Flat on 5 20.0
P2 of Corning 384 Well Plate 112 µL Flat on 5 20.0
A3 of Corning 384 Well Plate 112 µL Flat on 5 15.0
B3 of Corning 384 Well Plate 112 µL Flat on 5 15.0
C3 of Corning 384 Well Plate 112 µL Flat on 5 15.0
D3 of Corning 384 Well Plate 11

### Buffer B:

Complex command:

In [52]:
if not pip_20.hw_pipette['has_tip']:
    pip_20.pick_up_tip()

t_start = time.time()    
    
pip_20.distribute(vols_B_384,
              buffer_B["pos"],
              wells_to_pipette_to,
              new_tip='never',
              touch_tip=True, 
              disposal_volume = 1,
             )
t_end = time.time()
print(f'Needed {t_end-t_start}')

Needed 1.990797996520996


In [ ]:
pip_20.drop_tip()

In [53]:
t_start = time.time() 

for i, vol  in enumerate(vols_B_384):
    
    pip_20.aspirate(vol, buffer_B["pos"])
    
    pip_20.dispense(vol, wells_to_pipette_to[i])
    
    pip_20.move_to(reservoir["A12"].top())
    
    pip_20.blow_out(reservoir["A12"].bottom())
    
t_end = time.time()
print(f'Needed {t_end-t_start}')

Needed 2.0264580249786377


In [ ]:
protocol.home()

## Protein to source wells

We need to fill the first column with protein buffer starting from the second row:

In [54]:
plate.columns()[0][1:]

[B1 of Corning 384 Well Plate 112 µL Flat on 5,
 C1 of Corning 384 Well Plate 112 µL Flat on 5,
 D1 of Corning 384 Well Plate 112 µL Flat on 5,
 E1 of Corning 384 Well Plate 112 µL Flat on 5,
 F1 of Corning 384 Well Plate 112 µL Flat on 5,
 G1 of Corning 384 Well Plate 112 µL Flat on 5,
 H1 of Corning 384 Well Plate 112 µL Flat on 5,
 I1 of Corning 384 Well Plate 112 µL Flat on 5,
 J1 of Corning 384 Well Plate 112 µL Flat on 5,
 K1 of Corning 384 Well Plate 112 µL Flat on 5,
 L1 of Corning 384 Well Plate 112 µL Flat on 5,
 M1 of Corning 384 Well Plate 112 µL Flat on 5,
 N1 of Corning 384 Well Plate 112 µL Flat on 5,
 O1 of Corning 384 Well Plate 112 µL Flat on 5,
 P1 of Corning 384 Well Plate 112 µL Flat on 5]

In [55]:
if not pip_300.hw_pipette['has_tip']:
    pip_300.pick_up_tip()

t_start = time.time()  
    
pip_300.distribute(protein["v1"],
                  buffer_B["pos"],
                  plate.columns()[0][1:], # start at second row, in first row is only protein
                  touch_tip = True,
                  new_tip = 'never'
              )

t_end = time.time()
print(f'Needed {t_end-t_start}')

# pip_300.return_tip()

Needed 0.22037196159362793


In [22]:
pip_300.pick_up_tip()

<InstrumentContext: p300_single_v2.1 in RIGHT>

In [56]:
if not pip_300.hw_pipette['has_tip']:
    pip_300.pick_up_tip()

for source_well, dest_well in zip(plate.columns()[0][1:-1], plate.columns()[0][2:len(plate.columns())+1]):
    print(f'Transferring {protein["v_t"]} from {source_well} to {dest_well} \n')
    pip_300.aspirate(protein["v_t"], source_well)
    pip_300.touch_tip()
    pip_300.dispense(protein["v_t"], dest_well)

pip_300.aspirate(protein["v_t"])
pip_300.blow_out(protocol.fixed_trash['A1'])

t_end = time.time()
print(f'Needed {t_end-t_start}')
    
pip_300.drop_tip()

Transferring 50.23801187686225 from B1 of Corning 384 Well Plate 112 µL Flat on 5 to C1 of Corning 384 Well Plate 112 µL Flat on 5 

Transferring 50.23801187686225 from C1 of Corning 384 Well Plate 112 µL Flat on 5 to D1 of Corning 384 Well Plate 112 µL Flat on 5 

Transferring 50.23801187686225 from D1 of Corning 384 Well Plate 112 µL Flat on 5 to E1 of Corning 384 Well Plate 112 µL Flat on 5 

Transferring 50.23801187686225 from E1 of Corning 384 Well Plate 112 µL Flat on 5 to F1 of Corning 384 Well Plate 112 µL Flat on 5 

Transferring 50.23801187686225 from F1 of Corning 384 Well Plate 112 µL Flat on 5 to G1 of Corning 384 Well Plate 112 µL Flat on 5 

Transferring 50.23801187686225 from G1 of Corning 384 Well Plate 112 µL Flat on 5 to H1 of Corning 384 Well Plate 112 µL Flat on 5 

Transferring 50.23801187686225 from H1 of Corning 384 Well Plate 112 µL Flat on 5 to I1 of Corning 384 Well Plate 112 µL Flat on 5 

Transferring 50.23801187686225 from I1 of Corning 384 Well Plate 112 

<InstrumentContext: p300_single_v2.0 in LEFT>

In [ ]:
for line in protocol.commands():
    print(line)

## Protein to destination wells

Now transfer protein to different salt concentrations:

# Q: change tips?

In [28]:
if not pip_20.hw_pipette['has_tip']:
    pip_20.pick_up_tip()

# for more careful protein handling:
pip_20.flow_rate.aspirate = 2
pip_20.flow_rate.dispense = 2

for row in plate.columns_by_name()['1'][0:2]:
    pip_20.distribute(v_protein_to_wells,
                  plate.columns_by_name()['1'],
                  list(np.flip(wells_to_pipette_to)), # <-- note that
                  touch_tip = True,
                  new_tip = 'never',
                  mix_after = (3, 20),
                  )
#     pip_20.drop_tip()

KeyboardInterrupt: 

In [27]:
plate.columns_by_name()['1'][0:2]

[A1 of Corning 384 Well Plate 112 µL Flat on 5,
 B1 of Corning 384 Well Plate 112 µL Flat on 5]

In [ ]:
for line in protocol.commands():
    print(line)

Notes:

- 12.01.22: I exchanged the p300 single for the p20 multi
- as a result, I had to calibrate pipette and tip length... (30 min)

- when using mix_[before/after] it will mix always. When you tell it to dispense 45 µl with a 20 µl pipette, it will pipette 20 µl -> mix, 20 µl -> mix and 5 µl -> mix

In [ ]:
df = pd.DataFrame({"Protein concentration in µM": protein_concentrations, "Wells": plate.columns()[1]})
df["Wells"] = df["Wells"].astype(str).str[0:2]
df

In [ ]:
df["Wells"].astype(str)

In [ ]:
plate.rows()

In [ ]:
protein_buffer["pos"]

In [30]:
pip_300.drop_tip()

<InstrumentContext: p300_single_v2.1 in RIGHT>

25 µl Buffer B in source wells.

50 µl transferred

In [ ]:
pip_20.pick_up_tip()

In [ ]:
pip_20.drop_tip()

In [ ]:
[method_name for method_name in dir(protocol)
                  if callable(getattr(protocol, method_name))]

In [ ]:
protocol.door_closed

In [ ]:
protocol.home()

In [ ]:
pip_20.move_to(reservoir["A12"].top())

In [ ]:
pip_20.move_to(reservoir["A12"].bottom())

In [ ]:
reservoir["A12"].bottom().point

In [ ]:
point = reservoir["A12"].bottom().point
point.z += 1
point

In [ ]:
point

In [ ]:
(x,y,z) =  [c for c in reservoir["A2"].().point]

In [ ]:
z += 1
loc = Location(Point(x, y, z), None)

In [ ]:
last_concentration = protein["c"]

v_buffer  = (8*num_cols)

# we need more total volume than transfer volume
v_src_well = (v_buffer) + (1 * protein["transfer volume"])

# include the original concentration
c_src_wells = [protein["c"]]

# calculate the concentration in each source well
for r in range(1, len(plate.rows())):
    c = last_concentration * (protein["transfer volume"] / v_src_well)
    c_src_wells.append(c)
    
    last_concentration = c

In [ ]:
protein_concentrations = [c_src_well * protein["transfer volume to dest"] / (max_volume_buffers + protein["transfer volume to dest"]) for c_src_well in c_src_wells]

In [ ]:
print("Protein concentrations in source wells:")
[print(f'{c:.2e} µl') for c in c_src_wells]

print("Protein concentrations in destination wells")
[print(f'{c:.2e} µM') for c in protein_concentrations];

In [26]:
pip_300.pick_up_tip()

<InstrumentContext: p300_single_v2.1 in RIGHT>

In [25]:
pip_300.return_tip()

<InstrumentContext: p300_single_v2.1 in RIGHT>

In [40]:
pip_20.aspirate(20, buffer_B["pos"])

<InstrumentContext: p20_multi_v2.1 in LEFT>

In [31]:
protocol.home()

In [29]:
pip_20.drop_tip()

<InstrumentContext: p20_multi_v2.1 in LEFT>

Since we use a 384 well plate, we need to accomodate for that: